In [1]:
# importing the necessary packages

import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
import scipy

# importing the dataset
data_permits = pd.read_csv('Historical_DOB_Permit_Issuance.csv')
data_popn = pd.read_csv('2010+Census+Population+By+Zipcode+(ZCTA).csv')

/Users/devanshimittal/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (8,9,10,15,31,33,36,51) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Question 1

What fraction of all construction permits in this data set correspond to renewed permits? The filing type is recorded under column 'Filing Status'. For first-time permits the value in this column is 'INITIAL' and for renewed permits it is 'RENEWAL'.

In [3]:
# assuming that each record in the dataset is a unique construction permit. 

print("The fraction of all construction permits that are renewed are", \
          data_permits[data_permits['Filing Status'] == 'RENEWAL'].shape[0]/data_permits.shape[0])

The fraction of all construction permits that are renewed are 0.23756920864755


## Question 2

The filing type of a permit is recorded under column 'Filing Status'. For first-time permits the value in this column is 'INITIAL' and for renewed permits it is 'RENEWAL'. What kind of an owner the permit was issued to is recorded in the column "Owner's Business Type", for corporations the value is 'CORPORATION', for individuals it is 'INDIVIDUAL'. If you consider only renewed permits, what is the ratio of the number of permits issued to corporations to the number of permits issued to individuals?

In [4]:
# getting only renewed permits
data_renewed_permits = data_permits[data_permits['Filing Status'] == 'RENEWAL'] 
# getting corporation permits counts
number_crporations = data_renewed_permits[data_renewed_permits["Owner's Business Type"]=='CORPORATION'].shape[0]
# getting individual permits counts
number_individuals = data_renewed_permits[data_renewed_permits["Owner's Business Type"]=='INDIVIDUAL'].shape[0]

In [5]:
print("Considering only renewed permits, the ratio of number of permits issued to corporations vs to individuals is",\
      number_crporations/number_individuals)

Considering only renewed permits, the ratio of number of permits issued to corporations vs to individuals is 1.0805818771914573


## Question 3

Consider only those permits that were issued for more than 365 days. The date on which a permit was issued is recorded in column 'Issuance Date', the date the permit expires is in column 'Expiration Date'. What fraction of these at least year-long permits were issued in the borough with the highest number of such permits? The borough where the construction will take place is in column 'BOROUGH'.

In [106]:
data_permits['number_of_days_issued'] = pd.to_datetime(data_permits['Expiration Date']) - \
pd.to_datetime(data_permits['Issuance Date'])

# getting more than a year permits only
data_permits_morethan365 = data_permits[data_permits['number_of_days_issued']> '365 days']

# fraction of total year long permits permits by Borough
dt = pd.DataFrame(data_permits_morethan365['BOROUGH'].value_counts()/len(data_permits_morethan365))

In [83]:
print("The fraction of at least year long permits with highest number of permits was",  max(dt['BOROUGH']),\
      "in the borough", dt[dt['BOROUGH']==max(dt['BOROUGH'])].index[0])

The fraction of at least year long permits with highest number of permits was 0.44938589413557717 in the borough MANHATTAN


## Question 4

Limit your analysis to permits that were filed in 2010. The column recording the date of permit filing is called 'Filing Date'. For each ZIP code compute the ratio between the number of permits issued in 2010 and number of residents in that ZIP code. Make sure to discard any ZIP codes with population lower than 1000 people. Note: you will need to use both the DOB permits dataset and the US Census dataset. In the DOB permits dataset, the ZIP code is recorded under column 'Postcode'. How many ZIP codes are outliers in terms of number of construction permits per number of people - more specificaly, for how many ZIP codes does this computed ratio exceed the mean by more than twice the standard deviation?


In [ ]:
permits_2010 = data_permits[pd.to_datetime(data_permits['Filing Date']).dt.year == 2010]

In [84]:
# discarding less than 1000
popn_morethan1000 = data_popn[data_popn['2010 Census Population'] >= 1000]
permits_popn_byZip = pd.merge(permits_2010, popn_morethan1000,  left_on = "Postcode", right_on = 'Zip Code ZCTA')

# number of permits per post code
permit_cnt_byZip = permits_popn_byZip[['BOROUGH','Postcode']].groupby('Postcode').count().reset_index()

# population per zip code
popn_count_byZip = permits_popn_byZip[['Zip Code ZCTA', '2010 Census Population']].groupby('Zip Code ZCTA').mean().reset_index()

permits_popn_data = pd.merge(permit_cnt_byZip, popn_count_byZip,  left_on = "Postcode", right_on = 'Zip Code ZCTA')

# permit counts/ population count by zip code
permits_popn_data['ratio'] = permits_popn_data['BOROUGH']/permits_popn_data['2010 Census Population'] 

# getting only outliers data
outliers = permits_popn_data[(permits_popn_data['ratio'] - permits_popn_data['ratio'].mean()) > 2 * permits_popn_data['ratio'].std()]

print("The number of zip codes that are outliers are", outliers.shape[0])


The number of zip codes that are outliers are 9


# Question 5

Observe how the number of issued permits changes accross the years. Limit your analysis to years 1990-2012 (both inclusive). What is the coefficient of determination (R squared) between the year a permit was issued and the number of issued permits that year? The column recording the date when a permit was issued is called 'Issuance Date'.

In [107]:
permits_90_12 = data_permits[pd.to_datetime(data_permits['Issuance Date']).dt.year.isin(range(1990, 2013))]
permits_90_12['permit issue year'] =pd.to_datetime(permits_90_12['Issuance Date']).dt.year

/Users/devanshimittal/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [53]:
#number of permits per year
permits_by_year = permits_90_12[['BOROUGH','permit issue year']].groupby('permit issue year').count().reset_index()

In [54]:
# x = year, y = number of permits by year
slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(permits_by_year['permit issue year'], permits_by_year['BOROUGH'])

In [55]:
print("The coefficient of determination between the year a permit was issued and number of issued permits that year", \
      r_value**2)

The coefficient of determination between the year a permit was issued and number of issued permits that year 0.9068202122816982


## Question 6

Let's investigate how the number of construction jobs that start in the summer vs the winter changes across the years. The date of construction job start is recorded in column 'Job Start Date'. For every year, compute the ratio between the number of construction jobs that start in the peak of summer (in July and August) and the number of jobs that start in the peak of winter (in January and February). Again limit your analysis to years 1990-2012 (both inclusive). Find the year when this ratio was maximal - what was the value of this ratio for that year?

In [119]:
permits_90_12['Job Start Date Months'] = pd.to_datetime(permits_90_12['Job Start Date'], \
                                                                  errors = 'coerce').dt.month

/Users/devanshimittal/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [108]:
permits_90_12['Job Start Date Months'] = pd.to_datetime(permits_90_12['Job Start Date'], \
                                                                  errors = 'coerce').dt.month
#number of permits for summer - moonths 7 and 8
summer_permits = permits_90_12[['permit issue year', 'Job Start Date Months', 'BOROUGH']]\
    [permits_90_12['Job Start Date Months'].isin(['7', '8'])].groupby('permit issue year').count().reset_index()

#number of permits for summer - moonths 1 and 2
winter_permits = permits_90_12[['permit issue year', 'Job Start Date Months', 'BOROUGH']]\
    [permits_90_12['Job Start Date Months'].isin(['1', '2'])].groupby('permit issue year').count().reset_index()

total_permits = pd.merge(summer_permits, winter_permits, on = 'permit issue year')

total_permits['ratio'] = total_permits['BOROUGH_x']/total_permits['BOROUGH_y']

print("The year that the ratio was maximal was", \
      total_permits[total_permits['ratio']==total_permits['ratio'].max()]['permit issue year'][0], "and the ratio was", 
      total_permits['ratio'].max())

/Users/devanshimittal/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


The year that the ratio was maximal was 1990 and the ratio was 3.498069498069498


##  Question 7

If we look at how permit duration varies with the construction start date, it appears like jobs that start in November tend to have shorter permit duration. The date of construction job start is recorded in column 'Job Start Date'. The date on which a permit was issued is recorded in column 'Issuance Date', the date the permit expires is in column 'Expiration Date'. Calculate the chi-square test statistic for testing whether a permit is more likely to be issued for less than 60 days when construction job start date is November. Again limit your analysis to data from years 1990-2012 (both inclusive).

In [109]:
# assuming we are comparing < 60 days vs >= 60 days for november vs not november from 1990-2012
# getting total counts < 60 days and >= 60 days for November vs Not November
contingency_table = [[permits_90_12[(permits_90_12['Job Start Date Months'] ==11) & 
                         (permits_90_12['number_of_days_issued']<'60 days')].shape[0], \
          permits_90_12[(permits_90_12['Job Start Date Months'] ==11) & 
                        (permits_90_12['number_of_days_issued']>='60 days')].shape[0]],
          [permits_90_12[(permits_90_12['Job Start Date Months'] !=11) & 
                         (permits_90_12['number_of_days_issued']<'60 days')].shape[0], \
          permits_90_12[(permits_90_12['Job Start Date Months'] !=11) & 
                        (permits_90_12['number_of_days_issued']>='60 days')].shape[0]]]

In [113]:
permits_90_12[(permits_90_12['Job Start Date Months'] !=11) & 
                         (permits_90_12['number_of_days_issued']<'60 days')].shape[0]/ \
          permits_90_12[(permits_90_12['Job Start Date Months'] !=11) ].shape[0]

0.06871749195823593

In [114]:
permits_90_12[(permits_90_12['Job Start Date Months'] !=11) & 
                         (permits_90_12['number_of_days_issued']>='60 days')].shape[0]/ \
          permits_90_12[(permits_90_12['Job Start Date Months'] !=11)].shape[0]

0.931281134946019

In [115]:
permits_90_12[(permits_90_12['Job Start Date Months'] ==11) & 
                         (permits_90_12['number_of_days_issued']<'60 days')].shape[0]/ \
          permits_90_12[(permits_90_12['Job Start Date Months'] ==11)].shape[0]

0.10951440158053447

In [118]:
permits_90_12[(permits_90_12['Job Start Date Months'] ==11) & 
                         (permits_90_12['number_of_days_issued']>='60 days')].shape[0]/ \
          permits_90_12[(permits_90_12['Job Start Date Months'] ==11)].shape[0]

0.8904803992929188

In [101]:
stat, p, dof, expected = chi2_contingency(contingency_table)

In [102]:
print("The chi-square test statistic for testing whether a permit is more likely to be issued for less than 60 days when construction job start date is November compared to not in November is", stat)

The chi-square test statistic for testing whether a permit is more likely to be issued for less than 60 days when construction job start date is November compared to not in November is 4401.959208420233
